<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/00_polars_pandas_time.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [3]:
#!pip install polars
#! pip install pyarrow

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/ab/8e/01084c7c42885e05729d34d696dd17c30bf51467588265ed843fc37c7be9/polars-0.20.19-cp38-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 824.5 kB/s eta 0:00:0000:0100:04


In [4]:
import pandas as pd
import polars as pl
import numpy as np


In [5]:
# Configuramos el número de filas
num_rows = 10_000_000

# Creamos un DataFrame de pandas
df_pandas = pd.DataFrame({
    "A": np.random.rand(num_rows),
    "B": np.random.randint(0, 100, num_rows)
})

# Convertimos el DataFrame de pandas a un DataFrame de Polars
df_polars = pl.from_pandas(df_pandas)


# Operaciones a comparar

In [6]:
import timeit


## Filtrado

In [7]:

# Filtrado en pandas
def filter_pandas():
    return df_pandas[df_pandas["A"] < 0.5]

# Filtrado en Polars
def filter_polars():
    return df_polars.filter(pl.col("A") < 0.5)

time_pandas_filter = timeit.timeit(filter_pandas, number=1)
time_polars_filter = timeit.timeit(filter_polars, number=1)

print(f"Pandas Filtrado: {time_pandas_filter} segundos")
print(f"Polars Filtrado: {time_polars_filter} segundos")


Pandas Filtrado: 0.18282816698774695 segundos
Polars Filtrado: 0.02515737502835691 segundos


## Agregacion

In [35]:
# Agregación en pandas
def aggregate_pandas():
    return df_pandas.groupby("B").agg({"A": "mean"})

# Agregación en Polars
def aggregate_polars():
    return df_polars.groupby("B").agg(pl.col("A").mean())

time_pandas_agg = timeit.timeit(aggregate_pandas, number=1)
time_polars_agg = timeit.timeit(aggregate_polars, number=1)

print(f"Pandas Agregación: {time_pandas_agg} segundos")
print(f"Polars Agregación: {time_polars_agg} segundos")


Pandas Agregación: 0.08750266605056822 segundos
Polars Agregación: 0.021555625018663704 segundos


/var/folders/_m/zrywqjcn66j88_lz9f9pm4gc0000gn/T/ipykernel_50959/870000003.py:7: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  return df_polars.groupby("B").agg(pl.col("A").mean())


## Operaciones Columna

In [36]:
# Operaciones de columna en pandas
def column_ops_pandas():
    df_pandas["C"] = df_pandas["A"] + df_pandas["B"]

# Operaciones de columna en Polars
def column_ops_polars():
    df_polars.with_columns((pl.col("A") + pl.col("B")).alias("C"))

time_pandas_col_ops = timeit.timeit(column_ops_pandas, number=1)
time_polars_col_ops = timeit.timeit(column_ops_polars, number=1)

print(f"Pandas Operaciones de Columna: {time_pandas_col_ops} segundos")
print(f"Polars Operaciones de Columna: {time_polars_col_ops} segundos")


Pandas Operaciones de Columna: 0.01444624993018806 segundos
Polars Operaciones de Columna: 0.025740500073879957 segundos


# Guardar Datos

In [37]:
import os

In [38]:
def write_csv_pandas():
    df_pandas.to_csv("data_pandas.csv", index=False)

time_write_csv_pandas = timeit.timeit(write_csv_pandas, number=1)
print(f"Tiempo de escritura en CSV (pandas): {time_write_csv_pandas} segundos")

# Tamaño del archivo CSV
size_csv = os.path.getsize("data_pandas.csv")
print(f"Tamaño del archivo CSV: {size_csv} bytes")


Tiempo de escritura en CSV (pandas): 20.889748458983377 segundos
Tamaño del archivo CSV: 403523534 bytes


In [39]:
def write_parquet_polars():
    df_polars.write_parquet("data_polars.parquet")

time_write_parquet_polars = timeit.timeit(write_parquet_polars, number=1)
print(f"Tiempo de escritura en Parquet (Polars): {time_write_parquet_polars} segundos")

# Tamaño del archivo Parquet
size_parquet = os.path.getsize("data_polars.parquet")
print(f"Tamaño del archivo Parquet: {size_parquet} bytes")


Tiempo de escritura en Parquet (Polars): 0.3089657499222085 segundos
Tamaño del archivo Parquet: 86367136 bytes


## Leer Datos

In [40]:
def read_csv_pandas():
    pd.read_csv("data_pandas.csv")

time_read_csv_pandas = timeit.timeit(read_csv_pandas, number=1)
print(f"Tiempo de lectura desde CSV (pandas): {time_read_csv_pandas} segundos")


Tiempo de lectura desde CSV (pandas): 2.179799208068289 segundos


In [41]:
def read_parquet_polars():
    pl.read_parquet("data_polars.parquet")

time_read_parquet_polars = timeit.timeit(read_parquet_polars, number=1)
print(f"Tiempo de lectura desde Parquet (Polars): {time_read_parquet_polars} segundos")


Tiempo de lectura desde Parquet (Polars): 0.16172245796769857 segundos
